In [7]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time

# Load base overlay (for GPIO control)
ol = Overlay('base.bit')  # Default PYNQ-Z2 overlay

# Initialize GPIO for IR output (adjust pin as needed)
IR_PIN = 0  # PMODA[0] (Pin 0 of GPIO)
ir_gpio = AxiGPIO(ol.ip_dict['leds_gpio']).channel1

# NEC Protocol Parameters
CARRIER_FREQ = 38000  # 38 kHz
BIT_TIME = 0.0005625  # 562.5 µs (NEC bit duration)
CLOCK_PERIOD = 1 / (2 * CARRIER_FREQ)  # 38 kHz toggle time

# NEC Functions
def send_nec_pulse(duration):
    """Generate a modulated NEC pulse (38 kHz carrier)"""
    end_time = time.time() + duration
    while time.time() < end_time:
        ir_gpio.write(1, 0x1 << IR_PIN)  # HIGH
        time.sleep(CLOCK_PERIOD)
        ir_gpio.write(0, 0x1 << IR_PIN)  # LOW
        time.sleep(CLOCK_PERIOD)

def send_nec_space(duration):
    """Generate a LOW space (no modulation)"""
    ir_gpio.write(0, 0x1 << IR_PIN)  # LOW
    time.sleep(duration)

def send_nec_bit(bit):
    """Send a single NEC bit (1 or 0)"""
    send_nec_pulse(BIT_TIME)  # Always 562.5µs pulse
    if bit == 1:
        send_nec_space(BIT_TIME * 3)  # 1.6875ms space (logic '1')
    else:
        send_nec_space(BIT_TIME)      # 562.5µs space (logic '0')

def send_nec_command(address, command):
    """Send a full NEC IR command (address + command)"""
    # NEC frame: [9ms pulse, 4.5ms space, address, ~address, command, ~command]
    
    # 1. Start pulse (9ms pulse + 4.5ms space)
    send_nec_pulse(0.009)      # 9ms pulse
    send_nec_space(0.0045)     # 4.5ms space
    
    # 2. Send address + inverted address
    for bit in range(8):
        send_nec_bit((address >> bit) & 0x1)
    for bit in range(8):
        send_nec_bit(~(address >> bit) & 0x1)
    
    # 3. Send command + inverted command
    for bit in range(8):
        send_nec_bit((command >> bit) & 0x1)
    for bit in range(8):
        send_nec_bit(~(command >> bit) & 0x1)
    
    # 4. Stop bit (562.5µs pulse)
    send_nec_pulse(BIT_TIME)
    ir_gpio.write(0, 0x1 << IR_PIN)  # Ensure LED is off

# Example: Send NEC commands
def send_ir_remote_code(button_code):
    """Map button codes to NEC commands (adjust as needed)"""
    nec_codes = {
        'power':  (0x00, 0x45),
        '+':   (0x00, 0x15),
        '-':   (0x00, 0x07),
        '1':      (0x00, 0x0C),
        '2':      (0x00, 0x18),
        '3':(0x00, 0x5e),
        '4':(0x00, 0x08),
        '5':(0x00, 0x1c),
        '6':(0x00, 0x5a),
        '7':(0x00, 0x42),
        '8':(0x00, 0x52),
        '9':(0x00, 0x4a),
    }
    if button_code in nec_codes:
        addr, cmd = nec_codes[button_code]
        send_nec_command(addr, cmd)
    else:
        print("Unknown command")

# Test: Send "4" command
send_ir_remote_code('4')